<span>
<b>Authors:</b> <a href="http://about.giuliorossetti.net">Giulio Rossetti</a>, <a href="https://andreafailla.github.io/">Andrea Failla</a><br/>
<b>Python version:</b>  >=3.7<br/>
<b>Networkx version:</b>  >=2.3<br/>
<b>Last update:</b> 03/07/2025
</span>

<a id='top'></a>
# Ties Strength & Resilience*

In this notebook are introduced some of the main analysis discussed in lecture 7.

**Note:** this notebook is purposely not 100% comprehensive, it only discusses the basic things you need to get started.

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

Reading Game of Thrones Season 6 edge data and defining the graph g (useful for further operations)

In [ ]:
!git clone https://github.com/sna-unipi/SNA-lectures-notebooks.git
!mkdir asioaf
!cp -r SNA-lectures-notebooks/data/asioaf/* asioaf/
!rm -rf SNA-lectures-notebooks

In [ ]:
def read_net_w(filename):
    g = nx.Graph()
    with open(filename) as f:
        f.readline()
        for l in f:
            l = l.split(",")
            g.add_edge(l[0], l[1], weight=int(l[2]))
    return g

# Game of Thrones data
season = 6
g = read_net_w(f'asioaf/got-s{season}-edges.csv')

## Bridges and Local Bridges

Checking if graph has bridges

In [ ]:
nx.has_bridges(g)

Generating all bridges in the graph

In [ ]:
list(nx.bridges(g))

Generating all local bridges in the graph and computing the span (i.e., the shortest path length between the endpoints if the local bridge is removed)

In [ ]:
list(nx.local_bridges(g, with_span=True))

In [ ]:
# add to edges
for u, v, span in nx.local_bridges(g, with_span=True):
    g[u][v]['span'] = span

## Tie Strength

Measuring Tie Strength for each pair of nodes in the graph through Neighborhood Overlap

In [ ]:
def neighborhood_overlap(G, u, v):
    """
    Compute overlap O_uv = |N(u) ∩ N(v)| / |N(u) ∪ N(v)|
    (excluding u,v themselves).
    """
    nbrs_u = set(G[u]) - {v}
    nbrs_v = set(G[v]) - {u}
    inter = nbrs_u & nbrs_v
    union = nbrs_u | nbrs_v
    return len(inter) / len(union) if union else 0.0


In [ ]:
for u, v in g.edges():
    g[u][v]['overlap'] = neighborhood_overlap(g, u, v)

Plotting the KDE (Kernel Density Estimation) of Neighborhood Overlap

In [ ]:
weights = [e[-1]['overlap'] for e in g.edges(data=True)]
pd.DataFrame(weights)[0].plot.kde()
plt.xlabel("Neighborhood Overlap")
plt.ylabel("Density")
plt.xlim(0,1)
plt.show()

Plotting the KDE (Kernel Density Estimation) of Interactions Weights

In [ ]:
weights_got = [d['weight'] for u,v,d in g.edges(data=True)]
pd.DataFrame(weights_got)[0].plot.kde()
plt.xlabel("Interaction Weights")
plt.ylabel("Density")
plt.xlim(0,max(weights_got))
plt.show()

## Network Resilience
In the following we:
- generate Random and Scale-free Network
- compute Reslience for both Networks (through molloy_reed and breakdown_threshold)
- plot Game of Thrones graph Resilience under Random Failure and Targeted Attacks

Generating Networks

In [ ]:
# Random
er = nx.erdos_renyi_graph(1000, 0.001)
# Scale-free
ba = nx.barabasi_albert_graph(1000, 2)

Computing node degree for both ER and BA Networks

In [ ]:
er_degs = list(dict(er.degree()).values())
ba_degs = list(dict(ba.degree()).values())

Defining Molloy-Reed threshold

In [ ]:
def molloy_reed(degrees):
    return (np.mean(degrees)*(np.mean(degrees)+1))/np.mean(degrees)

def breakdown_threshold(degrees):
    K = molloy_reed(degrees)
    return 1 - (1/(K-1))

Random network

In [ ]:
molloy_reed(er_degs)

In [ ]:
breakdown_threshold(er_degs)

Scale-free network

In [ ]:
molloy_reed(ba_degs)

In [ ]:
breakdown_threshold(ba_degs)

### Random Failures

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def random_node(g): # select a random node from graph
    return [np.random.choice(g.nodes())]

def dismantle(g, function, **args): # incrementally removes node from a graph and computes size of connected components
    total_nodes = g.number_of_nodes()
    removed_nodes = []
    components = []
    while len(g.nodes()) > 1:
        n = function(g, **args)[0]
        g.remove_node(n)
        removed_nodes.append((len(removed_nodes)+1)/total_nodes)
        comps = list(nx.connected_components(g))
        g_size = 0
        if len(comps)>0:
            g_size  = max([len(c)for c in comps])/total_nodes
        components.append(g_size)
    return removed_nodes, components

def get_sorted_nodes(g, score, reverse=True): # sort nodes
    nodes = score(g)
    if isinstance(nodes, dict):
        nodes = [(k, v) for k, v in nodes.items()]
    srt = sorted(nodes, key = lambda k: k[1], reverse = reverse)
    return [x[0] for x in srt]

def plot_dismantle(x, y, ax=None, **kwargs):
    """
    Plot dismantling results on a given axis or create new plot.

    Parameters:
    -----------
    x : list or array
        X-axis data (removed nodes)
    y : list or array
        Y-axis data (giant component size)
    ax : matplotlib.axes.Axes, optional
        Axis to plot on. If None, creates new figure
    **kwargs : dict
        Additional keyword arguments passed to plot function
    """
    if ax is None:
        fig, ax = plt.subplots()
        show_plot = True
    else:
        show_plot = False

    ax.plot(x, y, **kwargs)
    ax.set_xlabel("Removed Nodes")
    ax.set_ylabel("Giant Component size")

    if show_plot:
        plt.show()


Random Failure:
- Giant component size has a consistent decrease when a big fraction of nodes is removed

In [ ]:
h = g.copy()
rn, comps = dismantle(h, random_node)
plot_dismantle(rn, comps)

### Targeted Attacks

Targeted Node attack (hubs are removed first)
- Giant component size has a consistent decrease when a small fraction of nodes is removed

In [ ]:
h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.degree)
plot_dismantle(rn, comps)

Targeted Edge Attack (based on Edge Betweenness Centrality)
- Giant component size has a consistent decrease when a small fraction of nodes is removed

In [ ]:
h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.betweenness_centrality)
plot_dismantle(rn, comps)

Targeted Edge Attack (based on Edge Harmonic Centrality)
- Giant component size has a consistent decrease when a small fraction of nodes is removed

In [ ]:
h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.harmonic_centrality)
plot_dismantle(rn, comps)

## Exercises

### Tie strength over time
Load the Game of Thrones networks for different seasons (seasons 1-6)
- For each season, compute the neighborhood overlap for all edges
- Create visualizations comparing the distribution of tie strengths across seasons

## Failures and attacks on different topologies
* Generate three types of networks with the same number of nodes (500):
   - Erdős-Rényi random graph
   - Barabási-Albert scale-free graph
   - Watts-Strogatz small-world graph


* For each network:
   - Calculate the Molloy-Reed threshold
   - Compute the breakdown threshold
   - Simulate random failures (random node removal) and targeted attacks (high-degree first)
   - Plot the giant component size vs. fraction of removed nodes